In [1]:
LOAD_MODEL = 'models/attention.01.pt'
MODEL_CHECKPOINT = 'models/attention.01.pt'
DATASET_PATH = 'data/interim/preprocessed_paranmt3.tsv'

In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader

import os
os.chdir("..") # go to the root dir

# Get the Dataset

In [3]:
MAX_SENT_SIZE = 10
MAX_TOKENS = 25_000

In [4]:
from src.data.make_dataset import ParanmtDataset

train_dataset = ParanmtDataset(
    path=DATASET_PATH,
    max_sent_size=MAX_SENT_SIZE,
    train=True,
    seed=42,
)

In [5]:
train_dataset.build_vocab(
    min_freq=2,
    specials=['<unk>', '<pad>', '<sos>', '<eos>'],
    max_tokens=MAX_TOKENS,
)

In [6]:
enc_vocab = train_dataset.toxic_vocab
dec_vocab = train_dataset.neutral_vocab

print("size of encoder vocab:", len(enc_vocab))
print("size of decoder vocab:", len(dec_vocab))

size of encoder vocab: 12137
size of decoder vocab: 14544


In [7]:
val_dataset = ParanmtDataset(
    path=DATASET_PATH,
    max_sent_size=MAX_SENT_SIZE,
    vocabs=(enc_vocab, dec_vocab), # avoid data leakage
    train=False,
    seed=42,
    take_first=10_000,
)

In [8]:
train_dataset.df

,similarity,lenght_diff,toxic_sent,neutral_sent,toxic_val,neutral_val
0,0.784351,0.190476,"[dressing, like, a, bum, ?]","[walk, like, a, bum, ?]",0.911522,0.040193
1,0.610479,0.000000,"[you, bum, !]","[slacker, !]",0.968336,0.006487
2,0.866624,0.060606,"[chloe, ,, stop, being, so, paranoid, .]","[chloe, ,, stop, being, so, paranoid, .]",0.611291,0.021283
3,0.949912,0.157895,"[it, stinks, in, here, .]","[stinks, in, here, .]",0.698503,0.013900
4,0.885716,0.347826,"[i, want, to, silence, you, .]","[i, silence, you, .]",0.921562,0.020940
...,...,...,...,...,...,...
162956,0.816062,0.076923,"[that, is, a, stupid, excuse, !]","[that, is, a, cowardly, excuse, !]",0.999643,0.042241
162957,0.756084,0.300000,"[what, the, hell, ?, huh, ?]","[what, is, wrong, ?]",0.825482,0.000042
162958,0.618064,0.333333,"[like, your, pecker, .]","[like, a, cue, .]",0.978997,0.000061
162959,0.687292,0.037037,"[my, friend, i, asexual, beast, .]","[my, girl, i, asexual, bestie, .]",0.997259,0.071518


In [9]:
len(train_dataset), len(val_dataset)

(162961, 10000)

# Build the Dataloaders

In [10]:
batch_size = 128

In [11]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
)

In [12]:
# let's check if shape and everything is ok
for batch in train_dataloader:
    toxic_sent, neutral_sent = batch
    print("toxic_sent.shape:", toxic_sent.shape)
    print("neutral_sent.shape:", neutral_sent.shape)
    break

toxic_sent.shape: torch.Size([128, 10])
neutral_sent.shape: torch.Size([128, 10])


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Load the Model

- EncoderDecoder (Seq2Seq) with Attention mechanism

In [14]:
from src.models.attention.encoder import Encoder
from src.models.attention.decoder import Decoder
from src.models.attention.attention import Attention
from src.models.attention import Seq2SeqAttention

In [15]:
## Encoder
enc_input_dim = len(enc_vocab)
enc_embed_dim = 128
enc_hidden_dim = 256
enc_dropout = 0.4
enc_padding_idx = enc_vocab['<pad>']

## Decoder
dec_output_dim = len(dec_vocab)
dec_embed_dim = 128
dec_hidden_dim = 256
dec_dropout = 0.4
dec_padding_idx = dec_vocab['<pad>']

In [16]:
# load the encoder and decoder for our model
encoder = Encoder(
    input_dim=enc_input_dim,
    embed_dim=enc_embed_dim,
    hidden_dim=enc_hidden_dim,
    dec_hidden_dim=dec_hidden_dim,
    dropout=enc_dropout,
    vocab=enc_vocab,
    padding_idx=enc_padding_idx,
).to(device)

attention = Attention(
    enc_hidden_dim,
    dec_hidden_dim,
)

decoder = Decoder(
    output_dim=dec_output_dim,
    embed_dim=dec_embed_dim,
    hidden_dim=dec_hidden_dim,
    attention=attention,
    enc_hidden_dim=enc_hidden_dim,
    dropout=dec_dropout,
    vocab=dec_vocab,
    padding_idx=dec_padding_idx,
).to(device)

In [17]:
best_loss = float('inf')

model = Seq2SeqAttention(
    encoder=encoder,
    decoder=decoder,
    device=device,
    max_sent_size=MAX_SENT_SIZE,
).to(device)

In [18]:
from src.models.utils import count_parameters

print(f"number of parameters in model: {count_parameters(model)//1e6}M")

number of parameters in model: 74.0M


In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(ignore_index=dec_vocab['<pad>'])

In [19]:
from src.models.train_model import train

best_loss = train(
    model=model,
    loaders=(train_dataloader, val_dataloader),
    optimizer=optimizer,
    criterion=criterion,
    epochs=20,
    device=device,
    best_loss=best_loss,
    ckpt_path=MODEL_CHECKPOINT,
    clip_grad=1,
    teacher_force={
        'value': 1,
        'gamma': 0,
        'update_every_n_epoch': 5,
    } # first 10 epoch teacher force 1, after it will be turned off
)

Evaluating 5: 100%|██████████| 79/79 [00:02<00:00, 35.83it/s, loss=5.1] 


Update teacher force to 0


Evaluating 20: 100%|██████████| 79/79 [00:02<00:00, 36.22it/s, loss=3.46]


In [20]:
# let's load the model and predict
model = torch.load(MODEL_CHECKPOINT)
model.to(device)
model.eval()

Seq2SeqAttention(
  (encoder): Encoder(
    (vocab): Vocab()
    (embedding): Embedding(12137, 128, padding_idx=1)
    (rnn): GRU(128, 256, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.4, inplace=False)
  )
  (decoder): Decoder(
    (vocab): Vocab()
    (embedding): Embedding(14544, 128, padding_idx=1)
    (rnn): GRU(640, 256, batch_first=True)
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (dropout): Dropout(p=0.4, inplace=False)
    (fc_out): Sequential(
      (0): Linear(in_features=896, out_features=2048, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.4, inplace=False)
      (3): Linear(in_features=2048, out_features=4096, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.4, inplace=False)
      (6): Linear(in_features=4096, out_features=14544, bias=True)
    )
  )
)

In [25]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenizer = TreebankWordDetokenizer()

# let's see how our model works
num_examples = 10
num_sentence = 3
dataset = train_dataset
for _ in range(num_examples):
    idx = np.random.randint(0, len(dataset))
    toxic_sent = detokenizer.detokenize(dataset.df.loc[idx, 'toxic_sent'])
    neutral_sent = detokenizer.detokenize(dataset.df.loc[idx, 'neutral_sent'])
    
    print('toxic_sent:', toxic_sent)
    print('neutral_sent:', neutral_sent)
    preds = model.predict(
        toxic_sent,
        beam=True,
        beam_search_num_candidates=num_sentence,
        post_process_text=False,
        use_encoder_out=True,
    ) # let's use beam search
    print("predictions:")
    for i in range(num_sentence):
        print(f"\t{i+1})", preds[i])
    print("\n")

toxic_sent: fucking mosquitoes.
neutral_sent: (mosquitoes buzzing)
predictions:
	1) ['a', '.', '.', '<eos>']
	2) ['a', '.', '.', '.', '<eos>']
	3) ['a', '.', '.', '.', '.', '<eos>']


toxic_sent: you criminals go to jail.
neutral_sent: you outlaws are going to prison.
predictions:
	1) ['you', 'are', 'going', 'to', '.', '.', '<eos>']
	2) ['you', 'are', 'going', 'to', '.', '.', '.', '<eos>']
	3) ['you', 'are', 'going', 'to', '.', '.', '.', '.', '<eos>']


toxic_sent: you are masons.
neutral_sent: you are a mosey.
predictions:
	1) ['you', 'are', '.', '.', '<eos>']
	2) ['you', 'are', 'a', '.', '<eos>']
	3) ['you', 'are', 'are', '.', '<eos>']


toxic_sent: how the hell are you?
neutral_sent: how you doing?
predictions:
	1) ['how', 'are', 'you', '?', '?', '<eos>']
	2) ['how', 'are', 'you', '?', '<eos>']
	3) ['how', 'are', 'you', '?', '?', '?', '<eos>']


toxic_sent: i do not kill him.
neutral_sent: i am not killing it.
predictions:
	1) ['i', 'do', 'not', 'kill', 'him', '.', '<eos>']
	2) ['i'